## Loading Data

In [ ]:
import zipfile
with zipfile.ZipFile("/content/novozymes-enzyme-stability-prediction.zip", "r") as zip_ref:
  zip_ref.extractall("Data")

In [1]:
import pandas as pd
train_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv")

In [2]:
train_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5


In [3]:
train_updated_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")
train_updated_df.head()

,seq_id,protein_sequence,pH,data_source,tm
0,69,NaN,NaN,NaN,NaN
1,70,NaN,NaN,NaN,NaN
2,71,NaN,NaN,NaN,NaN
3,72,NaN,NaN,NaN,NaN
4,73,NaN,NaN,NaN,NaN


## Updated CSV

In [ ]:
import pandas as pd


df_train = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv", index_col="seq_id")
df_train_updates = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv", index_col="seq_id")

all_features_nan = df_train_updates.isnull().all("columns")

drop_indices = df_train_updates[all_features_nan].index
df_train = df_train.drop(index=drop_indices)

swap_ph_tm_indices = df_train_updates[~all_features_nan].index
df_train.loc[swap_ph_tm_indices, ["pH", "tm"]] = df_train_updates.loc[swap_ph_tm_indices, ["pH", "tm"]]

In [ ]:
df_train_updates

In [ ]:
df_train

In [2]:
df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train.csv")
updates = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/train_updates_20220929.csv")

drop_seq_ids = updates[updates["pH"].isnull()]["seq_id"].values
change_seq_ids = updates[updates["pH"].notnull()]["seq_id"].values

# drop
df.drop(df[df.seq_id.isin(drop_seq_ids)].index, inplace=True)

# change
df.loc[df.seq_id.isin(change_seq_ids), ["pH", "tm"]] = updates[
    updates.seq_id.isin(change_seq_ids)
][["pH", "tm"]].values

df.to_csv("updated_train.csv", index=False)

In [ ]:
df_updated = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Notebooks/updated_train.csv")

In [ ]:
df_updated.head()

## EDA and Preprocessing

### BioPandas

In [7]:
import biopandas

In [8]:
somethin = ppdb.df["ATOM"]

NameError: name 'ppdb' is not defined

In [ ]:
wt = somethin.groupby('residue_number').b_factor.agg('first').values

### Continue

In [4]:
df_test = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/novozymes-enzyme-stability-prediction/test.csv")
df_test.head()

,seq_id,protein_sequence,pH,data_source
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


In [44]:
df_updated = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Notebooks/updated_train.csv")
df_updated.drop("data_source", inplace = True, axis = 1)
df_updated.head()

,seq_id,protein_sequence,pH,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5


In [45]:
df_updated.isnull().sum()

seq_id                0
protein_sequence      0
pH                  286
tm                    0
dtype: int64

In [46]:
fillVal = df_updated[["pH"]].mean().round()
df_updated.fillna(fillVal, inplace = True)

In [47]:
df_updated.isnull().sum()

seq_id              0
protein_sequence    0
pH                  0
tm                  0
dtype: int64

### Encoding the Amino Acid Sequences

### Try One

##### Hashing Vectorizer

In [38]:
random_sequence = df_updated.iloc[1]["protein_sequence"]

In [31]:
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = HashingVectorizer(n_features = 2**4)

X = vectorizer.fit_transform(random_sequence)

In [32]:
X

<1x16 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

#### TfidfVectorizer

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()

X_ = vec.fit_transform(random_sequence)

In [34]:
vec.get_feature_names_out()

array(['aaadgeplhneeeragagqvgrslpqeseeqrtgsrprrrrdlgsrlqaqrraqrvawedgdenvgqtvipaqeeegiekpaevhptgkigakklrkleekqarkaqreaeeaereerkrlesqreaewkkeeerlrlkeeqkeeeerkaqeeqarreheeylklkeafvveeegvsetmteeqshsfltefinyikkskvvlledlafqmglrtqdainriqdlltegtltgviddrgkfiyitpeelaavanfirqrgrvsitelaqasnsliswgqdlpaqas'],
      dtype=object)

### Try Two

#### List Strategy (Encoding singular letters)

In [9]:
import numpy as np

In [11]:
random_sequence

'AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSRLQAQRRAQRVAWEDGDENVGQTVIPAQEEEGIEKPAEVHPTGKIGAKKLRKLEEKQARKAQREAEEAEREERKRLESQREAEWKKEEERLRLKEEQKEEEERKAQEEQARREHEEYLKLKEAFVVEEEGVSETMTEEQSHSFLTEFINYIKKSKVVLLEDLAFQMGLRTQDAINRIQDLLTEGTLTGVIDDRGKFIYITPEELAAVANFIRQRGRVSITELAQASNSLISWGQDLPAQAS'

In [12]:
random_seq_list = np.array(list(random_sequence))

In [35]:
random_seq_list

array(['A', 'A', 'A', 'D', 'G', 'E', 'P', 'L', 'H', 'N', 'E', 'E', 'E',
       'R', 'A', 'G', 'A', 'G', 'Q', 'V', 'G', 'R', 'S', 'L', 'P', 'Q',
       'E', 'S', 'E', 'E', 'Q', 'R', 'T', 'G', 'S', 'R', 'P', 'R', 'R',
       'R', 'R', 'D', 'L', 'G', 'S', 'R', 'L', 'Q', 'A', 'Q', 'R', 'R',
       'A', 'Q', 'R', 'V', 'A', 'W', 'E', 'D', 'G', 'D', 'E', 'N', 'V',
       'G', 'Q', 'T', 'V', 'I', 'P', 'A', 'Q', 'E', 'E', 'E', 'G', 'I',
       'E', 'K', 'P', 'A', 'E', 'V', 'H', 'P', 'T', 'G', 'K', 'I', 'G',
       'A', 'K', 'K', 'L', 'R', 'K', 'L', 'E', 'E', 'K', 'Q', 'A', 'R',
       'K', 'A', 'Q', 'R', 'E', 'A', 'E', 'E', 'A', 'E', 'R', 'E', 'E',
       'R', 'K', 'R', 'L', 'E', 'S', 'Q', 'R', 'E', 'A', 'E', 'W', 'K',
       'K', 'E', 'E', 'E', 'R', 'L', 'R', 'L', 'K', 'E', 'E', 'Q', 'K',
       'E', 'E', 'E', 'E', 'R', 'K', 'A', 'Q', 'E', 'E', 'Q', 'A', 'R',
       'R', 'E', 'H', 'E', 'E', 'Y', 'L', 'K', 'L', 'K', 'E', 'A', 'F',
       'V', 'V', 'E', 'E', 'E', 'G', 'V', 'S', 'E', 'T', 'M', 'T

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(min_df=0)

X__ = vec.fit_transform(random_seq_list)

#### LabelEncoder

In [14]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [15]:
X____ = encoder.fit_transform(random_seq_list)

##### Binarizer

In [ ]:
from sklearn.preprocessing import Binarizer

In [ ]:
binarizer = Binarizer()

X____b = binarizer.fit_transform(X____.reshape(-1,1))

In [ ]:
X____b # Doesn't work

##### Normalize

In [57]:
from sklearn.preprocessing import Normalizer

norm = Normalizer()

X____j = norm.fit_transform(X____.reshape(-1,1))

In [58]:
X____j

array([[0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [16]:
X____.max()

18

In [17]:
amino_acid_sequence = X____/18

In [1]:
amino_acid_sequence

NameError: name 'amino_acid_sequence' is not defined

#### Function that does the above for us (that works but isn't pratical)

In [9]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

def encode_sequence(sequence):

    encoder = LabelEncoder()

    random_sequence = np.array(list(sequence))
    random_encoded_sequence = encoder.fit_transform(random_sequence)
    random_encoded_sequence = random_encoded_sequence/random_encoded_sequence.max()
    
    return random_encoded_sequence



    
    # for n in len(df_updated["protein_sequence"]):
    #     random_sequence = df_updated.iloc[n]["protein_sequence"]
    #     random_sequence = np.array(list(random_sequence))
    #     random_encoded_sequence = encoder.fit_transform(random_sequence)
    #     random_encoded_sequence = random_encoded_sequence/random_encoded_sequence.max()

    #     return random_encoded_sequence

### Engineering Features

#### Finding Mean

In [19]:
from statistics import mean

In [57]:
df_updated

,seq_id,protein_sequence,pH,tm,average
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello
...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello


In [55]:
sequence = df_updated["protein_sequence"][1]

In [56]:
mean(sequence)

TypeError: can't convert type 'str' to numerator/denominator

In [48]:
df_updated["average"] = "hello"

In [50]:
df_updated

,seq_id,protein_sequence,pH,tm,average
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello
...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello


In [51]:
df_updated["average"] = [mean(v) for v in df_updated["protein_sequence"].values]

TypeError: can't convert type 'str' to numerator/denominator

In [52]:
df_updated

,seq_id,protein_sequence,pH,tm,average
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello
...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello


#### Length Column

In [37]:
df_updated["len"] = df_updated["protein_sequence"].str.len()

#### ProteinAnalysis from biopython

In [14]:
df_updated

,seq_id,protein_sequence,pH,tm
0,0,"[0.0, 0.0, 0.0, 0.0, 0.42105263157894735, 0.0,...",7.0,75.7
1,1,"[0.0, 0.0, 0.0, 0.05555555555555555, 0.2222222...",7.0,50.5
2,2,"[0.0, 0.0, 0.0, 0.21052631578947367, 0.7894736...",7.0,40.5
3,3,"[0.0, 0.0, 0.0, 0.7894736842105263, 0.26315789...",7.0,47.2
4,4,"[0.0, 0.0, 0.0, 0.8421052631578947, 0.42105263...",7.0,49.5
...,...,...,...,...
28976,31385,"[1.0, 1.0, 0.5263157894736842, 1.0, 0.78947368...",7.0,51.8
28977,31386,"[1.0, 1.0, 0.5789473684210527, 0.1052631578947...",7.0,37.2
28978,31387,"[1.0, 1.0, 0.6842105263157895, 0.7368421052631...",7.0,64.6
28979,31388,"[1.0, 1.0, 0.7894736842105263, 0.2105263157894...",7.0,50.7


In [30]:
df_updated.corr()

,seq_id,pH,tm,average,len
seq_id,1.000000,-0.009309,-0.002663,-0.023293,-0.017527
pH,-0.009309,1.000000,0.027573,-0.087561,0.107266
tm,-0.002663,0.027573,1.000000,-0.185351,-0.137504
average,-0.023293,-0.087561,-0.185351,1.000000,0.067922
len,-0.017527,0.107266,-0.137504,0.067922,1.000000


In [28]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [58]:
df_updated["molecular_weight"] = df_updated["protein_sequence"].apply(lambda x: ProteinAnalysis(x).molecular_weight())

In [60]:
df_updated["aromaticity"] = df_updated["protein_sequence"].apply(lambda x: ProteinAnalysis(x).aromaticity())

In [61]:
df_updated

,seq_id,protein_sequence,pH,tm,average,molecular_weight,aromaticity
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,75.7,hello,36320.7244,0.058651
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,50.5,hello,32837.9931,0.041958
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,40.5,hello,53428.8034,0.080483
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,47.2,hello,29475.5996,0.071698
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,49.5,hello,158761.9814,0.065472
...,...,...,...,...,...,...,...
28976,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,51.8,hello,61997.6230,0.089253
28977,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,37.2,hello,52637.6897,0.087420
28978,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,64.6,hello,14203.9225,0.117188
28979,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,50.7,hello,64367.8724,0.074199


#### Lambda Expression?

In [ ]:
df_updated["protein_sequence"] = df_updated["protein_sequence"].map(lambda x: encode_sequence(x))